In this notebook, we will train some explainers based on pretrained models from torchvision

In [1]:

import ICE.Data
import ICE.ModelWrapper
import ICE.Explainer
import ICE.utils

import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

import numpy as np

This time, we will train a husky explainer. It's based on ILSVRC2012 dataset, using class No.248.

**We assume that you already have the ILSVRC2012 dataset.**

This includes the following steps:
1. Wrap the model with *get_feature*, *feature_predict*, and *predict* methods
2. Provide a loader with data in form of (x,y)
3. Train the Explainer

In [2]:
import os
ImageNet_path = '\dataset\ILSVRC2012'
ImageNet_classes = []
ImageNet_val = {i:[] for i in range(1000)}

with open(os.path.join(ImageNet_path,'synset_words.txt'),'r') as f:
    for line in f:
        tsplit = line.split('\n')[0].split(' ')
        path = os.path.join(ImageNet_path,'ILSVRC2012_img_train',tsplit[0])
        name = ' '.join(tsplit[1:])
        #print (path,name)
        
        ImageNet_classes.append((path,name))

with open(os.path.join(ImageNet_path,'val.txt'),'r') as f:
    for line in f:
        img,label = line.split('\n')[0].split(' ')
        
        ImageNet_val[int(label)].append(os.path.join(ImageNet_path,'ILSVRC2012_img_val',img))

def make_imgs(paths = [],labels = []):
    imgs = []
    for i,path in enumerate(paths):
        imgs += [(os.path.join(path,t),labels[i]) for t in os.listdir(path)]
    return imgs



In [3]:

target_classes = [248]
title = "248_husky"
n_components = 10
classes_names = [' '.join([str(idx),ImageNet_classes[idx][1]]) for idx in target_classes]
layer_name = 'layer4'

print ("title:{}".format(title))
print ("target_classes:{}".format(target_classes))
print ("classes_names:{}".format(classes_names))
print ("n_components:{}".format(n_components))
print ("layer_name:{}".format(layer_name))

title:248_husky
target_classes:[248]
classes_names:['248 Eskimo dog, husky']
n_components:10
layer_name:layer4


In [4]:

loaders = [DataLoader(
        ICE.Data.ImageDataset(make_imgs([ImageNet_classes[label][0]],[label]),
                transforms.Compose([
                transforms.Resize((224,224)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                        std=[0.229, 0.224, 0.225]),
                ])),
        batch_size = 16,
        num_workers=8,
        #shuffle=True
) for label in target_classes]


m = models.resnet50(pretrained=True)
m = m.cuda()
m.eval()
model = ICE.ModelWrapper.PytorchModelWrapper(m,batch_size=16,predict_target=target_classes,input_channel_first = False,model_channel_first = True)

In [5]:

print ("-------------------------------------------------------------------------------------------")

try:
    shutil.rmtree('Explainers/'+title)
except:
    pass
# create an Explainer
Exp = ICE.Explainer.Explainer(title = title,
                layer_name = layer_name,
                class_names = classes_names,
                utils = ICE.utils.img_utils(mode = "torch"),
                n_components = n_components,
                reducer_type = "NMF"
               )

# train reducer based on target classes
Exp.train_model(model,loaders)
# generate features 
Exp.generate_features(model, loaders)
# generate global explanations
Exp.global_explanations()
# save the explainer, use load to load it with the same title
Exp.save()


-------------------------------------------------------------------------------------------
Training reducer:
1/5 Featuer maps gathered.
dataset too big, train with 0.77 instances
loading complete, with size of (996, 7, 7, 2048)
2/5 Reducer trained, spent 24.13231134414673 s.
3/5 Error estimated, fidelity: [0.06015709].
4/5 Weight estimator initialized.
5/5 Weight estimated.
visulizing features:
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
loading training data
Done with class: 248 Eskimo dog, husky, 1/1
Generate explanations with fullset condition


In [6]:
from IPython.display import HTML, display
import os
fpath = os.path.join("Explainers",title,"GE")
paths = [os.path.join(fpath,fname) for fname in os.listdir(fpath)]
display(HTML("".join(["<table><tr>"]+["<td><img src='{}'></td>".format(path) for path in paths]+["</tr></table>"])))

""


In [7]:

target_classes = [243,282]
title = "dog_cat"
n_components = 10
classes_names = [' '.join([str(idx),ImageNet_classes[idx][1]]) for idx in target_classes]
layer_name = 'layer4'

print ("title:{}".format(title))
print ("target_classes:{}".format(target_classes))
print ("classes_names:{}".format(classes_names))
print ("n_components:{}".format(n_components))
print ("layer_name:{}".format(layer_name))

title:dog_cat
target_classes:[243, 282]
classes_names:['243 bull mastiff', '282 tiger cat']
n_components:10
layer_name:layer4


In [8]:

loaders = [DataLoader(
        ICE.Data.ImageDataset(make_imgs([ImageNet_classes[label][0]],[label]),
                transforms.Compose([
                transforms.Resize((224,224)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                        std=[0.229, 0.224, 0.225]),
                ])),
        batch_size = 16,
        num_workers=8,
        #shuffle=True
) for label in target_classes]


m = models.resnet50(pretrained=True)
m = m.cuda()
m.eval()
model = ICE.ModelWrapper.PytorchModelWrapper(m,batch_size=16,predict_target=target_classes,input_channel_first = False,model_channel_first = True)

In [9]:

print ("-------------------------------------------------------------------------------------------")

try:
    shutil.rmtree('Explainers/'+title)
except:
    pass
# create an Explainer
Exp = ICE.Explainer.Explainer(title = title,
                layer_name = layer_name,
                class_names = classes_names,
                utils = ICE.utils.img_utils(mode = "torch"),
                n_components = n_components,
                reducer_type = "NMF"
               )

# train reducer based on target classes
Exp.train_model(model,loaders)
# generate features 
Exp.generate_features(model, loaders)
# generate global explanations
Exp.global_explanations()
# save the explainer, use load to load it with the same title
Exp.save()


-------------------------------------------------------------------------------------------
Training reducer:
1/5 Featuer maps gathered.
dataset too big, train with 0.38 instances
loading complete, with size of (996, 7, 7, 2048)
2/5 Reducer trained, spent 20.493043184280396 s.
3/5 Error estimated, fidelity: [0.05004624 0.0560113 ].
4/5 Weight estimator initialized.
5/5 Weight estimated.
visulizing features:
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
loading training data
Done with class: 243 bull mastiff, 1/2
Done with class: 282 tiger cat, 2/2
Generate explanations with fullset condition


In [10]:
# generate local explanation for catdog.png
from PIL import Image
img_path = "catdog.png"
img = Image.open(img_path).convert("RGB")
x = transforms.ToTensor()(transforms.Resize(Exp.utils.fsize)(img))
x = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])(x).numpy()
x = np.transpose(x,(1,2,0)) #to channel_last
Exp.local_explanations(x,model)

In [11]:
from IPython.display import HTML, display
import os
fpath = os.path.join("Explainers",title,"explanations","0")
paths = [os.path.join(fpath,fname) for fname in os.listdir(fpath) if 'explanation' in fname]
display(HTML("".join(["<table><tr>"]+["<td><img src='{}'></td>".format(path) for path in paths]+["</tr></table>"])))

,
